In [ ]:
import cv2
import mediapipe
import numpy as np
import csv

# Tutorial from: https://techtutorialsx.com/2021/05/19/mediapipe-face-landmarks-estimation/

drawingModule = mediapipe.solutions.drawing_utils
faceModule = mediapipe.solutions.face_mesh

circleDrawingSpec = drawingModule.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))
lineDrawingSpec = drawingModule.DrawingSpec(thickness=1, color=(0, 255, 0))
cap = cv2.VideoCapture(0)
with faceModule.FaceMesh() as face:
    while cap.isOpened():
        ret, frame = cap.read()
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = face.process(image)

        try:
            overall_face = results.multi_face_landmarks

            for facial_landmarks in results.multi_face_landmarks:
                point_headers = []
                point_values = []

                for i in range(0, 468):
                    pt1 = facial_landmarks.landmark[i]
                    point_headers.append("x{}".format(i))
                    point_headers.append("y{}".format(i))
                    point_headers.append("z{}".format(i))

                    point_values.append(pt1.x)
                    point_values.append(pt1.y)
                    point_values.append(pt1.z)
                # persist to CSV

                 # Export to CSV
                with open('csvs/neutral_2021_11_20.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(point_values) 

                f.close()
        except:
            pass

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        if results.multi_face_landmarks != None:
            for faceLandmarks in results.multi_face_landmarks:
                drawingModule.draw_landmarks(image, faceLandmarks, faceModule.FACEMESH_CONTOURS, circleDrawingSpec,
                                            lineDrawingSpec)

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [15]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import pandas as pd

from custom.iris_lm_depth import from_landmarks_to_depth


In [3]:
LEFT_EYE_LANDMARKS_ID = np.array([33, 133])
RIGHT_EYE_LANDMARKS_ID = np.array([362, 263])

YELLOW = (0, 255, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
RED = (0, 0, 255)
SMALL_CIRCLE_SIZE = 1
LARGE_CIRCLE_SIZE = 2

POINTS_IDX = [33, 133, 362, 263, 61, 291, 199]
POINTS_IDX = list(set(POINTS_IDX))
POINTS_IDX.sort()

In [20]:
landmarks = None
smooth_left_depth = -1
smooth_right_depth = -1
smooth_factor = 0.1
cap = cv2.VideoCapture(0)

with mp.solutions.face_mesh.FaceMesh(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5,
) as face_mesh:
    while cap.isOpened():
        
        ret, frame = cap.read()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)
        try:
            image_width, image_height, image_channels = image.shape

            image_size = (image_height, image_width)
            focal_length = image_height

            multi_face_landmarks = results.multi_face_landmarks

            if multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = np.array(
                    [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
                )
                landmarks = landmarks.T

                (
                    left_depth,
                    left_iris_size,
                    left_iris_landmarks,
                    left_eye_contours,
                ) = from_landmarks_to_depth(
                    frame_rgb,
                    landmarks[:, LEFT_EYE_LANDMARKS_ID],
                    image_size,
                    is_right_eye=False,
                    focal_length=focal_length,
                )

                (
                    right_depth,
                    right_iris_size,
                    right_iris_landmarks,
                    right_eye_contours,
                ) = from_landmarks_to_depth(
                    frame_rgb,
                    landmarks[:, RIGHT_EYE_LANDMARKS_ID],
                    image_size,
                    is_right_eye=True,
                    focal_length=focal_length,
                )

                if smooth_right_depth < 0:
                    smooth_right_depth = right_depth
                else:
                    smooth_right_depth = (
                        smooth_right_depth * (1 - smooth_factor)
                        + right_depth * smooth_factor
                    )

                if smooth_left_depth < 0:
                        smooth_left_depth = left_depth
                else:
                    smooth_left_depth = (
                        smooth_left_depth * (1 - smooth_factor)
                        + left_depth * smooth_factor
                    )

                if landmarks is not None:

                    landmark_idx = 0
                    df_headers = []
                    df_values = []

                    # add subset of facemesh to dataframe
                    for ii in POINTS_IDX:

                        landmark = (landmarks[0, ii], landmarks[1, ii], landmarks[2, ii])
                        __add_landmark_to_df(landmark, landmark_idx, df_headers, df_values)

                        landmark_idx += 1

                    # add eye contours to dataframe
                    eye_landmarks = np.concatenate(
                        [
                            right_eye_contours,
                            left_eye_contours,
                        ]
                    )
                    for landmark in eye_landmarks:

                        __add_landmark_to_df(landmark, landmark_idx, df_headers, df_values)

                        landmark_idx += 1

                    # add iris landmarks to dataframe
                    iris_landmarks = np.concatenate(
                        [
                            right_iris_landmarks,
                            left_iris_landmarks,
                        ]
                    )
                    for landmark in iris_landmarks:

                        __add_landmark_to_df(landmark, landmark_idx, df_headers, df_values)

                        landmark_idx += 1

                    # create dataframe
                    df = pd.DataFrame([df_values], columns = df_headers)
                except:
                        pass

                __display_debug_image(landmarks, eye_landmarks, iris_landmarks, image, image_size)


In [18]:
def __display_debug_image(face_landmarks, eye_landmarks, iris_landmarks, image, image_size):
        """Displays image with iris landmarks"""

        if face_landmarks is not None:

            # draw subset of facemesh
            for ii in POINTS_IDX:
                pos = (np.array(image_size) * face_landmarks[:2, ii]).astype(np.int32)
                image = cv2.circle(image, tuple(pos), LARGE_CIRCLE_SIZE, GREEN, -1)

            # draw eye contours
            for landmark in eye_landmarks:
                pos = (np.array(image_size) * landmark[:2]).astype(np.int32)
                image = cv2.circle(image, tuple(pos), SMALL_CIRCLE_SIZE, RED, -1)

            # draw iris landmarks
            for landmark in iris_landmarks:
                pos = (np.array(image_size) * landmark[:2]).astype(np.int32)
                image = cv2.circle(image, tuple(pos), SMALL_CIRCLE_SIZE, YELLOW, -1)

        cv2.imshow('Debug Image', image)

        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [19]:
def __add_landmark_to_df(landmark, landmark_idx, df_headers, df_values):
    """Helper function that adds a landmark to the dataframe"""

    df_headers.append("x{}".format(landmark_idx))
    df_headers.append("y{}".format(landmark_idx))
    df_headers.append("z{}".format(landmark_idx))

    df_values.append(landmark[0])
    df_values.append(landmark[1])
    df_values.append(landmark[2])


In [ ]:
import argparse

import cv2
import mediapipe as mp
import numpy as np

from custom.iris_lm_depth import from_landmarks_to_depth
from videosource import FileSource, WebcamSource

mp_face_mesh = mp.solutions.face_mesh

points_idx = [33, 133, 362, 263, 61, 291, 199]
points_idx = list(set(points_idx))
points_idx.sort()

left_eye_landmarks_id = np.array([33, 133])
right_eye_landmarks_id = np.array([362, 263])

dist_coeff = np.zeros((4, 1))

YELLOW = (0, 255, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
RED = (0, 0, 255)
SMALL_CIRCLE_SIZE = 1
LARGE_CIRCLE_SIZE = 2


def main(inp):
    if inp is None:
        frame_height, frame_width = (720, 1280)
        source = WebcamSource(width=frame_width, height=frame_height)
    else:
        source = FileSource(inp)
        frame_width, frame_height = (int(i) for i in source.get_image_size())

    image_size = (frame_width, frame_height)

    # pseudo camera internals
    focal_length = frame_width

    landmarks = None
    smooth_left_depth = -1
    smooth_right_depth = -1
    smooth_factor = 0.1

    with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as face_mesh:

        for idx, (frame, frame_rgb) in enumerate(source):
            results = face_mesh.process(frame_rgb)
            multi_face_landmarks = results.multi_face_landmarks

            if multi_face_landmarks:
                face_landmarks = results.multi_face_landmarks[0]
                landmarks = np.array(
                    [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]
                )
                landmarks = landmarks.T

                (
                    left_depth,
                    left_iris_size,
                    left_iris_landmarks,
                    left_eye_contours,
                ) = from_landmarks_to_depth(
                    frame_rgb,
                    landmarks[:, left_eye_landmarks_id],
                    image_size,
                    is_right_eye=False,
                    focal_length=focal_length,
                )

                (
                    right_depth,
                    right_iris_size,
                    right_iris_landmarks,
                    right_eye_contours,
                ) = from_landmarks_to_depth(
                    frame_rgb,
                    landmarks[:, right_eye_landmarks_id],
                    image_size,
                    is_right_eye=True,
                    focal_length=focal_length,
                )

                if smooth_right_depth < 0:
                    smooth_right_depth = right_depth
                else:
                    smooth_right_depth = (
                        smooth_right_depth * (1 - smooth_factor)
                        + right_depth * smooth_factor
                    )

                if smooth_left_depth < 0:
                    smooth_left_depth = left_depth
                else:
                    smooth_left_depth = (
                        smooth_left_depth * (1 - smooth_factor)
                        + left_depth * smooth_factor
                    )

                print(
                    f"depth in cm: {smooth_left_depth / 10:.2f}, {smooth_right_depth / 10:.2f}"
                )
                print(f"size: {left_iris_size:.2f}, {right_iris_size:.2f}")

            if landmarks is not None:

                # draw subset of facemesh
                for ii in points_idx:
                    pos = (np.array(image_size) * landmarks[:2, ii]).astype(np.int32)
                    frame = cv2.circle(frame, tuple(pos), LARGE_CIRCLE_SIZE, GREEN, -1)

                # draw eye contours
                eye_landmarks = np.concatenate(
                    [
                        right_eye_contours,
                        left_eye_contours,
                    ]
                )
                for landmark in eye_landmarks:
                    pos = (np.array(image_size) * landmark[:2]).astype(np.int32)
                    frame = cv2.circle(frame, tuple(pos), SMALL_CIRCLE_SIZE, RED, -1)

                # draw iris landmarks
                iris_landmarks = np.concatenate(
                    [
                        right_iris_landmarks,
                        left_iris_landmarks,
                    ]
                )
                for landmark in iris_landmarks:
                    pos = (np.array(image_size) * landmark[:2]).astype(np.int32)
                    frame = cv2.circle(frame, tuple(pos), SMALL_CIRCLE_SIZE, YELLOW, -1)

                # write depth values into frame
                depth_string = "{:.2f}cm, {:.2f}cm".format(
                    smooth_left_depth / 10, smooth_right_depth / 10
                )
                frame = cv2.putText(
                    frame,
                    depth_string,
                    (50, 50),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    GREEN,
                    2,
                    cv2.LINE_AA,
                )

            source.show(frame)
